In [1]:
import os
import shutil

import geopandas as gpd
import nivapy3 as nivapy

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Create GIS data package

Creates a package of GIS data layers clipped to a specified bounding box. Fill-in section 1 below and then run all cells.

## 1. User input

In [2]:
# Path to boundng box
bbox_path = "../examples/data_package/bounding_box.shp"

# Output folder
out_fold = "../examples/data_package"

# Zip arhive to create (omit .zip extension)
# NOT the same as 'out_fold'
zip_path = "../examples/glitre_gis"

# Vector layers of interest
# See https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md
vec_layers = [
    "physical.norway_nibio_corine_2018_poly",
    "physical.norway_nve_elvis_river_network_line",
    "physical.norway_nve_innsjo_poly",
]

# Raster layers of interest
# See shared/01_datasets/spatial
ras_layers = [
    r"/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif",
    r"/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif",
]

## 2. Read bounding box

In [3]:
# Read clip features
bb_gdf = gpd.read_file(bbox_path)

## 3. Clip vector layers

Layers list [here](https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md).

In [4]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

Connection successful.


In [5]:
# Loop over layers
for layer in vec_layers:
    print(f"Processing: {layer}")

    # Read layer, clipping to bounding box
    schema, table = layer.split(".")
    gdf = nivapy.da.read_postgis(
        schema,
        table,
        eng,
        clip=bb_gdf,
    )

    # Save
    out_shp = os.path.join(out_fold, f"{table}.shp")
    gdf.to_file(out_shp)

Processing: physical.norway_nibio_corine_2018_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nve_elvis_river_network_line
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nve_innsjo_poly


<ipython-input-5-cae4ed073ccb>:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(out_shp)


## 4. Clip raster layers

In [6]:
# Loop over layers
for layer in ras_layers:
    print(f"Processing: {layer}")

    # Read layer, clipping to bounding box
    fname = os.path.split(layer)[1]
    out_gtiff = os.path.join(out_fold, fname)
    nivapy.spatial.clip_raster_to_gdf(
        layer,
        out_gtiff,
        bb_gdf,
        bounding_box=True,
    )

Processing: /home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif
Converting to the projection of target dataset ({'proj': 'utm', 'zone': 33, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})
Processing: /home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif
Converting to the projection of target dataset ({'proj': 'utm', 'zone': 33, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})


## 5. Create `.zip` archive

In [7]:
shutil.make_archive(zip_path, "zip", out_fold)

'/home/jovyan/development/dstoolkit_cookbook/examples/glitre_gis.zip'